In [122]:
import warnings
import itertools
import pickle
import copy
import math
import bisect
import torch
import json
import numpy as np
import pandas as pd
import torch.nn as nn
from transformers import BertModel
from itertools import combinations
from itertools import chain
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset


F_P = {
    0: [0,1],
    1: [2,3],
    2: [4,5,6,7,8,9,10,11],
    3: [12,13],
}
    

def generate_partition(f_part):  #input a dict
    setnum = len(f_part)  #子集數
    num_set = [i for i in range(setnum)]  #子集數list
    cardlist = [len(i) for i in f_part.values()]  #各子集cardinality
    f_list = [i for i in f_part.values()]  #list of list
    all_subsets = []
    for r in range(setnum+1):
        for s in combinations(num_set, r):
            tmp = []
            for t in s:
                tmp.extend(f_list[t])
            all_subsets.append(np.array(sorted(tmp)))
    # Create a hash table to store the index of each subset
    subset_index_table = {}
    for index, value in enumerate(all_subsets):
        subset_index_table[tuple(value)] = index
    return all_subsets, subset_index_table, len(all_subsets)


class Hypercube_wpart:
    '''
    A class to create a hypercube object which stores values of vertices
    '''    
    #輸入維度
    def __init__(self, partition):   #input a dict
        self.f_part = partition
        self.n_dim = len(self.f_part)
        self.elem_count = sum([len(i) for i in self.f_part.values()])
        # vertex_values is a dictionary to store the value of each vertex.
        # Because np.array is not hashable, we use tuple to represent the vertex.
        self.vertex_values = {}
        self.vertices, self.vertex_index, self.vertex_num = generate_partition(self.f_part)  #vertex 上的value一次考慮整個subset
        self.edges, self.edge_num = self.build_edges()
        self.differential_matrix = None
        self.weight_matrix = None
        self.generate_min_l2_norm_matrix()
    
    def build_edges(self):
        num_set = [i for i in range(self.n_dim)]  #子集數list
        s_set = set(num_set)  #轉集合
        cardlist = [len(i) for i in self.f_part.values()]  #各子集cardinality
        f_list = [i for i in self.f_part.values()]  #list of list
        #print(f'Receive {f_list}')
        s_subset = set(tuple(i) for i in f_list)
        edges = []
        for r in range(self.n_dim): 
            for v in combinations(num_set, r):
                v_set = set(v)
                adjunct_v = s_set - v_set
                for new_elem in adjunct_v:
                    d_set = v_set | {new_elem}
                    outlist, inlist = [],[]                    
                    for k in v_set:
                        outlist.extend(f_list[k])
                    for l in d_set:
                        inlist.extend(f_list[l])
                    edges.append(((np.array(sorted(outlist))),np.array(sorted(inlist))))
        return edges, len(edges)
    
    def get_elements(self, index):
        return tuple(self.f_part[index])

    def set_vertex_values(self, vertex_values):         #設置點值
        for v in vertex_values:                         #用鍵值來做查找
            self.vertex_values[v] = vertex_values[v]
        
    def does_edge_exist(self, v1, v2):
        if abs(len(v1)-len(v2))==1:
            interset = np.intersect1d(v1,v2)
            smaller = v1 if len(v1)<len(v2) else v2
            return True if np.array_equal(smaller, interset) else False
        else:
            return False
    
    # Establish the matrix A in the above formula: AX-Y
    def generate_differential_matrix(self):
        if self.differential_matrix is None:
            self.differential_matrix = np.zeros((self.edge_num+1, self.vertex_num))
            for i,v_pair in enumerate(self.edges):
                j = self.vertex_index[tuple(v_pair[1])]
                k = self.vertex_index[tuple(v_pair[0])]
                self.differential_matrix[i][j] = 1
                self.differential_matrix[i][k] = -1
            # Add one more equestion that x_0 = 0 into the matrix form
            self.differential_matrix[-1][0]=1
        return self.differential_matrix

    # Pre-calcuate "W=(A^T*A)^-1*A^T" for the formula "X = ((A^T*A)^-1*A^T)*Y
    def generate_min_l2_norm_matrix(self):
        matrix_A = self.generate_differential_matrix()
        matrix_A_T = np.transpose(matrix_A)
        self.weight_matrix = np.linalg.inv(matrix_A_T @ matrix_A) @ matrix_A_T

    def get_gradient_vector(self):
        gradient_vector = np.zeros(self.edge_num)
        for i,v_pair in enumerate(self.edges):
            gradient_vector[i] = self.vertex_values[tuple(v_pair[1])]-self.vertex_values[tuple(v_pair[0])]    
        return gradient_vector      
        
    def get_partial_gradient_vector(self,subset_i):  #feature->subset->allow input int or tuple
        if isinstance(subset_i, int):
            feature_i = self.get_elements(subset_i)
        else:
            feature_i = []            
            for i in subset_i:
                feature_i.append(self.get_elements(i))
            feature_i = tuple(chain.from_iterable(feature_i))
        partial_gradient_vector = np.zeros(self.edge_num)
        for i,v_pair in enumerate(self.edges):
            if (not set(feature_i).issubset(set(v_pair[0]))) and (set(feature_i).issubset(set(v_pair[1]))):
                partial_gradient_vector[i] = self.vertex_values[tuple(v_pair[1])]-self.vertex_values[tuple(v_pair[0])]    
        return partial_gradient_vector
    
    def resolve_vi(self, subset_i, phi_0=0):  #feature->subset
        pd = self.get_partial_gradient_vector(subset_i)
        # Append equation x_0=0 at the end of partial gradient vector.
        pd = np.append(pd, phi_0)
        vi = self.weight_matrix @ pd
        # Reconstruct the vertex values
        new_vertices = {}
        for i,v in enumerate(self.vertices):
            new_vertices[tuple(v)] = vi[i]
        return vi, new_vertices


with open('E_vi_202406020109.pkl', 'rb') as f:
    vertex_df = pickle.load(f)


In [123]:
print(vertex_df.shape)
print(vertex_df.tail())

(400, 16)
      ()    (0, 1)    (2, 3)  (4, 5, 6, 7, 8, 9, 10, 11)  (12, 13)  \
395  0.0 -0.064708 -0.005147                    0.101029 -0.007977   
396  0.0  0.055786 -0.020408                    0.059892 -0.034912   
397  0.0 -0.026820 -0.023915                    0.016681  0.003001   
398  0.0 -0.077867  0.017598                    0.081263 -0.020323   
399  0.0 -0.073594 -0.019889                    0.108478  0.007847   

     (0, 1, 2, 3)  (0, 1, 4, 5, 6, 7, 8, 9, 10, 11)  (0, 1, 12, 13)  \
395     -0.057073                          0.046742       -0.065423   
396      0.039505                          0.213909        0.051529   
397     -0.042432                         -0.050602       -0.024431   
398     -0.018893                          0.095185       -0.080381   
399     -0.101365                          0.056089       -0.072601   

     (2, 3, 4, 5, 6, 7, 8, 9, 10, 11)  (2, 3, 12, 13)  \
395                          0.098155       -0.015533   
396                         

In [124]:
#list of Hypercube_wpart and set vertex values
subsets, _, _ = generate_partition(F_P)
cubelist = []
for idx in range(vertex_df.shape[0]):
    tp = Hypercube_wpart(F_P)
    vertices = {}
    values = vertex_df.iloc[idx].tolist()
    for v in subsets:
        vertices[tuple(v)] = values.pop(0)
        tp.set_vertex_values(vertices)
    cubelist.append(tp)
  

In [125]:
#get partial gradient vector and residual for n*subsets
rows, cols = len(vertex_df), len(F_P)
pg_m = [[None for i in range(cols)] for j in range(rows)]  #partial gradient vector matrix in 3D
residual_m = [[None for i in range(cols)] for j in range(rows)]  #residual matrix in 3D
for i in range(rows):
    for j in range(cols):
        pd = cubelist[i].get_partial_gradient_vector(j)  #partial gradient of instance i of subset j 
        pg_m[i][j] = pd
        vi, new_vs = cubelist[i].resolve_vi(j)
        h1 = Hypercube_wpart(F_P)
        h1.set_vertex_values(new_vs)
        dvi = h1.get_gradient_vector()
        residual_m[i][j] = dvi-pd

In [126]:
#l2 norm for single subset
import pandas as pd
ptgd_l2 = [[float(np.linalg.norm(pg_m[i][j])) for j in range(cols)] for i in range(rows)]
r_l2 = [[float(np.linalg.norm(residual_m[i][j])) for j in range(cols)] for i in range(rows)]
scaled_norm = [[float(r_l2[i][j] / ptgd_l2[i][j]) for j in range(cols)] for i in range(rows)]
scn_df = pd.DataFrame(scaled_norm)
ptgd_df = pd.DataFrame(ptgd_l2)
r_df = pd.DataFrame(r_l2)
l2_mean = scn_df.mean()
l2_mean

0    0.417736
1    0.489972
2    0.322631
3    0.537737
dtype: float64

In [128]:
#get partial gradient vector and residual for n*length2subsets
length = 2
rows = len(vertex_df)
subset_num = len(F_P)
items = [i for i in range(subset_num)]
combo = [i for i in combinations(items,length)]
cols = len(combo)
pg_m = [[None for i in range(cols)] for j in range(rows)]  #partial gradient vector matrix in 3D
residual_m = [[None for i in range(cols)] for j in range(rows)]  #residual matrix in 3D
for i in range(rows):
    for j in range(cols):
        pg = cubelist[i].get_partial_gradient_vector(combo[j])  #partial gradient of instance i of subset j 
        pg_m[i][j] = pg
        vi, new_vs = cubelist[i].resolve_vi(combo[j])
        h1 = Hypercube_wpart(F_P)
        h1.set_vertex_values(new_vs)
        dvi = h1.get_gradient_vector()
        residual_m[i][j] = dvi-pg

#l2 norm 
import pandas as pd
ptgd_l2 = [[float(np.linalg.norm(pg_m[i][j])) for j in range(cols)] for i in range(rows)]
r_l2 = [[float(np.linalg.norm(residual_m[i][j])) for j in range(cols)] for i in range(rows)]
scaled_norm = [[float(r_l2[i][j] / ptgd_l2[i][j]) for j in range(cols)] for i in range(rows)]
scn_df = pd.DataFrame(scaled_norm, columns=combo)
ptgd_df = pd.DataFrame(ptgd_l2,columns=combo)
r_df = pd.DataFrame(r_l2)
l2_mean = scn_df.mean()
l2_mean

(0, 1)    0.520332
(0, 2)    0.465590
(0, 3)    0.577667
(1, 2)    0.531090
(1, 3)    0.597425
(2, 3)    0.541257
dtype: float64

In [120]:
0    0.417736
1    0.489972
2    0.322631
3    0.537737
dtype: float64

[[array([ 0.        ,  0.        ,  0.        ,  0.        , -0.02411681,
          0.        ,  0.        , -0.07407826,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        , -0.05087313,  0.        ,
         -0.02388185,  0.        , -0.1258972 ,  0.        , -0.07351732,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         -0.04654273, -0.13016245]),
  array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         -0.04903698,  0.        ,  0.        ,  0.        ,  0.        ,
         -0.10811901,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        , -0.0757933 ,  0.        ,  0.        ,  0.        ,
          0.        , -0.06453598, -0.1258972 ,  0.        ,  0.        ,
          0.        , -0.11441267,  0.        ,  0.        , -0.08719686,
          0.        , -0.13016245]),
  array([ 0.        ,  0.        ,  0.

{(): 0.0,
 (0, 1): -0.06606286764144897,
 (2, 3): -0.016101419925689697,
 (4, 5, 6, 7, 8, 9, 10, 11): -0.006980836391448975,
 (12, 13): -0.001281142234802246,
 (0, 1, 2, 3): -0.0901796817779541,
 (0, 1, 4, 5, 6, 7, 8, 9, 10, 11): -0.11509984731674194,
 (0, 1, 12, 13): -0.06875497102737427,
 (2, 3, 4, 5, 6, 7, 8, 9, 10, 11): -0.04007577896118164,
 (2, 3, 12, 13): -0.0191195011138916,
 (4, 5, 6, 7, 8, 9, 10, 11, 12, 13): -0.018878281116485596,
 (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11): -0.1659729778766632,
 (0, 1, 2, 3, 12, 13): -0.0926368236541748,
 (0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13): -0.13329094648361206,
 (2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13): -0.0496712327003479,
 (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13): -0.17983368039131165}

int